### Initialize ellipsoids with DPD

In [1]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
#from flowermd.library import EllipsoidForcefield, EllipsoidChain
#from flowermd.utils import get_target_box_number_density
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [131]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        density,
        bond_L=0.1,
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.density = density
        N=lengths*num_mols
        L = np.cbrt(N / self.density)
        print("L",L)
        self.L = L
        self.box = mb.Box(lengths=np.array([self.L] * 3))
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        rand_range = (self.L/2 - (self.lpar + (self.bond_L / 2)))
        print('range',rand_range)
        for i in range(length):
            translate_by = np.random.uniform(low=-rand_range, high=rand_range, size=(3,))
            translate_by /= np.linalg.norm(translate_by)
            translate_by *= self.bond_L
            print('normalization',translate_by)
            this_bead = mb.clone(bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
                print('pbcs',self.pbc(translate_by+last_bead.pos,box=([self.L]*3)))
                this_bead.translate(by=self.pbc(translate_by+last_bead.pos,box=([self.L]*3)))
            else:
                this_bead.translate(by=np.random.uniform(low=-rand_range, high=rand_range, size=(3,)))
            chain.add(this_bead)
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"
        #chain.box = mb.Box(lengths=np.array([self.L] * 3))
        return chain

    def pbc(self,d,box):
        for i in range(3):
            print('box[i]/2',box[i]/2)
            while d[i] > box[i]/2 or d[i] < -box[i]/2:
                if d[i] < -box[i]/2:
                    d[i] += box[i]
                if d[i] > box[i]/2:
                    d[i] -= box[i]
        return d

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=0.1,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

#write new system class to skip pack and shrink step
class RandomSystem(System):
    def __init__(self, molecules, base_units=dict()):
        self.L = molecules.L
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        comp = mb.Compound()
        comp.add(chain)
        comp.box = mb.Box(lengths=np.array([self.L] * 3))
        print(comp.box)
        return comp

In [132]:
ellipsoid_chain = EllipsoidChainRand(lengths=10,num_mols=10,lpar=1.0,bead_mass=1.0,density=0.1)
ff = DPD_FF(epsilon=1.0,lpar=1.0,lperp=0.5,A=100,gamma=1.0,kT=2.0,r_cut=1.94,bond_k=100,bond_r0=1.0,angle_k=30,angle_theta0=1.9)
ff.hoomd_forces
system = RandomSystem(molecules=ellipsoid_chain)
system.to_gsd('test.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=1.0,lperp=0.5
)
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

ellipsoid_sim.save_restart_gsd()
ellipsoid_sim.run_NVT(n_steps=1e4, kT=2.0, tau_kt=10*ellipsoid_sim.dt)
ellipsoid_sim.flush_writers()

L 10.0
range 3.95
normalization [ 0.03537977  0.05914564 -0.07245734]
normalization [ 0.05820659 -0.05759914 -0.05739627]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [ 0.05853703  0.76399051 -1.42792671]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [0.08148835 0.00931844 0.05720853]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [ 0.14002538  0.77330895 -1.10821818]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [-0.07119867  0.02410036 -0.06595394]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [ 0.06882671  0.79740931 -0.91167212]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [ 0.01056166 -0.09738825  0.02009926]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [ 0.07938837  0.70002106 -0.62907286]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [ 0.06557564 -0.06287343  0.04179435]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [ 0.144964    0.63714763 -0.32477851]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [0.03519242 0.05954369 0.07222217]
box[i]/2 5.0
box[i]

*Warning*: pos 0:-4.99966955185 -4.17841053009 -6.63303041458
*Warning*: fractional pos :3.30448150635e-05 0.082158946991 -0.163303041458
*Warning*: lo: -5 -5 -5
*Warning*: hi: 5 5 5
*Warning*: Not all particles were found inside the given box


box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [1.96777051 1.55726528 3.5512405 ]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [-0.06768474  0.06535105  0.03388239]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [1.90008577 1.62261633 3.8476229 ]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [ 0.06316043  0.07687018 -0.01008639]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [1.96324621 1.69948651 4.10003651]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [0.07714768 0.05127601 0.0376697 ]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [2.04039389 1.75076252 4.40020621]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [-0.09772857  0.00096708 -0.02117055]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [1.94266532 1.75172959 4.64153566]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [ 0.02964044 -0.08514432  0.04326533]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
pbcs [1.97230576 1.66658527 4.94730099]
box[i]/2 5.0
box[i]/2 5.0
box[i]/2 5.0
normalization [-0.0638131  -0.05599

RuntimeError: Error initializing ParticleData